In [1]:
import sys
sys.path.append('..')

from app.database import _get_database_async
from rethinkdb import query as r

In [44]:
async def example_for_loop_async():
    async with await _get_database_async() as conn:
        sensor_table = r.table('sensors')
        async for sensor in await sensor_table.run(conn):
            print(sensor)


async def example_as_list_async():
    async with await _get_database_async() as conn:
        sensor_table = r.table('sensors')
        sensors = list((await sensor_table.run(conn)).items)
        print(sensors)

[{'coordinates': None, 'id': '4df86a94-dac5-499e-95b1-9ee7b8f65963', 'node_id': '!833c2233', 'region_id': '57b2b4c2-6d5a-4c4e-8d3b-5b4f4f4b4d54', 'uplink': False, 'watershed': None}]


In [2]:
# playground to test the async database and write rethinkdb queries
async def main():
    async with await _get_database_async() as conn:
        # match await r.table("sensors").get('4df86a94-dac5-499e-95b1-9ee7b8f65963').update(
        #     {"coordinates": r.point(3, 2)},
        #     return_changes=True
        # ).run(conn):
        #     case {'changes': [{'new_val': sensor}]}:
        #         print(sensor)
        #     case _:
        #         print("No changes")

        # await r.table("sensors").get('4df86a94-dac5-499e-95b1-9ee7b8f65963').update(
        #     {"coordinates": None,
        #      "watershed": None
        #      },
        # ).run(conn)
        await r.table("regions").insert(
            {
                "id": "57b2b4c2-6d5a-4c4e-8d3b-5b4f4f4b4d54",
                "name": "Courand Family Ranch",
                "bottom_left": r.point(
                    -103.31498383879455,
                    30.275977600526915,
                ),
                "top_right": r.point(-103.1300, 30.487701),
            }
        ).run(conn)

await main()